# Update PIONEER custom vocabularies

In [ ]:
import getpass
import pandas as pd
import sqlalchemy as sa

from sqlalchemy import create_engine, text
from pathlib import Path

In [ ]:
pio_vocab_dir = Path('/home/jovyan/ohdsi-omop-pioneer/pioneer_custom_vocabulary/')

## Schema names

In [ ]:
VOCAB_SCHEMA = 'vocab'
TEMP_SCHEMA = 'temp_schema'

## Connect to the db

In [ ]:
server = 'postgresql'
port = '5432'
db = 'ohdsi'
engine = create_engine(f'postgresql://{input("User:")}:{getpass.getpass("Password:")}@{server}:{port}/{db}')

In [ ]:
sa.inspect(engine).get_schema_names()

## Load new custom vocab files into a temporary schema

In [ ]:
with engine.connect() as con:
    con.execute(f'CREATE SCHEMA {TEMP_SCHEMA};')

In [ ]:
# Custom vocab file paths
vocabulary_file = pio_vocab_dir/'pioneer_vocabulary.csv'
concept_class_file = pio_vocab_dir/'pioneer_concept_class.csv'
concept_file = pio_vocab_dir/'pioneer_concepts.csv'

# Currently loaded custom vocabulary records as dfs
query = f"SELECT * FROM {VOCAB_SCHEMA}.vocabulary WHERE vocabulary_concept_id = 0"
current_vocab_df = pd.read_sql(sql=query, con=engine)
query = f"SELECT * FROM {VOCAB_SCHEMA}.concept_class WHERE concept_class_concept_id = 0"
current_concept_class_df = pd.read_sql(sql=query, con=engine)
query = f"SELECT * FROM {VOCAB_SCHEMA}.concept WHERE concept_id >= 2000000000"
date_cols = ['valid_start_date', 'valid_end_date']
current_concept_df = pd.read_sql(sql=query, con=engine, parse_dates=date_cols)

# Read new custom vocab files as dfs
vocab_df = pd.read_csv(vocabulary_file, sep=',', dtype=current_vocab_df.dtypes.to_dict())
concept_class_df = pd.read_csv(concept_class_file, sep=',', dtype=current_concept_class_df.dtypes.to_dict())
dtypes = {k: v for k, v in current_concept_df.dtypes.to_dict().items() if k not in date_cols}
concept_df = pd.read_csv(concept_file, sep=',', dtype=dtypes, parse_dates=date_cols)

# Load the vocab files into the newly created schema
vocab_df.to_sql(name='vocabulary', con=engine, schema=TEMP_SCHEMA, index=False, if_exists='replace')
concept_class_df.to_sql(name='concept_class', con=engine, schema=TEMP_SCHEMA, index=False, if_exists='replace')
concept_df.to_sql(name='concept', con=engine, schema=TEMP_SCHEMA, index=False, if_exists='replace')

## Check for new custom vocabularies and replace the current ones

Each replacement query will be executed as a transaction.  
Meaning that if it fails it will automatically do a rollback, otherwise it will be committed.

In [ ]:
def df_a_is_subset_of_df_b(a: pd.DataFrame, b: pd.DataFrame) -> bool:
    return len(a.merge(b)) == len(a)

### vocabulary

In [ ]:
new_records_available = not df_a_is_subset_of_df_b(a=vocab_df, b=current_vocab_df)
print(f'New vocabulary table records available: {new_records_available}')

In [ ]:
replace_query = f"""
WITH ins1 AS (
    INSERT INTO {VOCAB_SCHEMA}.vocabulary
    SELECT * FROM {TEMP_SCHEMA}.vocabulary
)
DELETE FROM {VOCAB_SCHEMA}.vocabulary
WHERE vocabulary_concept_id = 0 
AND vocabulary_id in 
    (select vocabulary_id from {TEMP_SCHEMA}.vocabulary);
"""
with engine.begin() as con:
    con.execute(replace_query)

### concept_class

In [ ]:
new_records_available = not df_a_is_subset_of_df_b(a=concept_class_df, b=current_concept_class_df)
print(f'New concept_class table records available: {new_records_available}')

In [ ]:
replace_query = f"""
WITH ins1 AS (
    INSERT INTO {VOCAB_SCHEMA}.concept_class
    SELECT * FROM {TEMP_SCHEMA}.concept_class
)
DELETE FROM {VOCAB_SCHEMA}.concept_class 
WHERE concept_class_concept_id = 0;
"""
with engine.begin() as con:
    con.execute(replace_query)

### concept

In [ ]:
new_records_available = not df_a_is_subset_of_df_b(a=concept_df, b=current_concept_df)
print(f'New concept table records available: {new_records_available}')

In [ ]:
replace_query = f"""
WITH ins1 AS (
    INSERT INTO {VOCAB_SCHEMA}.concept
    SELECT * FROM {TEMP_SCHEMA}.concept
)
DELETE FROM {VOCAB_SCHEMA}.concept
WHERE concept_id in 
    (select concept_id from {TEMP_SCHEMA}.concept);
"""
with engine.begin() as con:
    con.execute(replace_query)

## Drop the temporary schema

In [ ]:
with engine.connect() as con:
    con.execute(f'DROP SCHEMA {TEMP_SCHEMA} CASCADE;')